# Numerical simulation of the wave equation in 2D using Finite Difference

## Import libraries

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as anim
from IPython.display import HTML
import sys

----

## Create domain model properties (i.e. speed of waves), and discretisation grid

In [ ]:
# 501 grid-points along the x-axis, 351 along the z-axis
nx = 501
nz = 351

# start with homogeneous model - same wave-speed everywhere.
c = np.full((nx,nz),1500.0)  # Note: 1500m/s is typical acoustic velocity of water

# a faster layer, deeper in model, to show reflection of waves when crossing sharp interface
c[:,140:] = 3000.0  # Note: 3000m/s is fairly typical acoustic velocity of sedimentary rock

print('Range of speeds:  %.1fm/s to %.1fm/s' % (c.min(),c.max()))

In [ ]:
length = 3500.0 # assign a length to the model in metres

dx = length/(nx-1) # calculate the spatial increment between model parameters

depth = dx*(nz-1) # calculate the depth from the length and number of x and z gridpoints

print('Domain is %d x %d grid-points (%.1fm x %.1fm)' % (nx,nz,length,depth))
print('Grid-spacing (dx) is %.2fm' % (dx))

In [ ]:
def plot_model(c):
    plt.figure(figsize=(10,6))
    plt.imshow(c.T) # plotting the velocity model (note transposed, to orient plot in way we expect)
    plt.colorbar()
    plt.xlabel('x gridpoints')
    plt.ylabel('z gridpoints')
    plt.title('Velocity model (m/s)')
    plt.show()

def plot_slice(c,xgrid):
    plt.figure(figsize=(7,6))
    plt.plot(c[xgrid,:],range(nz-1,-1,-1))  # note two arrays, 2nd one (range) gets model on x axis
    plt.xlabel('propagation speed / m/s')
    plt.ylabel('z gridpoints')
    plt.title('Vertical slice through model at x-gridpoint=%d' % xgrid)
    plt.show()

In [ ]:
plot_model(c)
plot_slice(c,200)

----
## Modelling constraints
#### CFL stability condition should be satisfied – the 'Courant number'.

The dimensionless Courant number, $C$, gives a measure of how far a signal will travel across a grid-cell (/between grid-points) within one time-step.
(Hence why I will very often refer to it as 'the crossing factor'.)

The crossing factor is $c$.$\delta t$/$\delta x$ for speed $c$, and we want the maximum value of this within the whole domain to satisfy some constraint that depends upon the finite-difference stencil(s) in use when modelling.

i.e. we want:$\quad \text{max}(c)$.$\delta t/\delta x <= C_{max}$, where $\ C_{max}$ depends on the discretisations.

We can turn this around to find the maximum time-step for our model and grid-spacing, given $C_{max}$ for our discretisations:

$$\delta t_{best} = \frac{C_{max}\ \delta x}{\text{max}(c)}$$

#### This spatial stencil (simple 2nd order), with 2nd-order time-stepping, requires the max crossing factor to (usually) be no more than sqrt(1/2)
(i.e. can only cross up to 70.7% of a cell in one time-step – we'll use 70% here...)

In [ ]:
courant = 0.70  # set the dimensionless max Courant number for 2nd-order FD grid in space & time

In [ ]:
time = 2.0  # desired length of the simulation in seconds
dt = (courant*dx)/c.max()  # define the best time-step by using the max Courant number
nt = int(time/dt+0.9999)  # how many steps are needed to cover that time
time = nt*dt  # turn that back into exact time for this number of steps
print('Time-step = %.5fs  Number of steps = %d  (Total time being modelled: %.5fs)' % (dt,nt,dt*nt))

### Create source function (Ricker wavelet with 8Hz peak frequency)

In [ ]:
# want to locate the source in the grid
sx = 150
sz = 80

The Ricker wavelet used here is a function of time that's defined from the second derivate of a Gaussian function (which is $G(t)=\text{e}^{-a^2 t^2}$).

–It's also known as the Mexican hat wavelet, due to its shape when plotted as a function of two variables.

It can be written so it is symmetric about time zero, with maximum at that time, ultimately decaying towards zero as $t$$\rightarrow$$±\infty$. However, we will shift it in time so that it starts near zero at our time zero.

After differentiating above Gaussian, $G(t)$, twice, flipping, then scaling (and before shifting in time), we get: $\quad R(t) = (1-2a^2 t^2)\ \text{e}^{-a^2 t^2}$

The peak frequency of the Ricker wavelet is at $f$=$a/\pi$.

The length of the wavelet, in time, before it decays close enough to zero for our purposes, is about $\frac{2.1}{f}$ (i.e. about $\frac{1.05}{f}$ seconds each side of the wavelet's central peak).

In [ ]:
def create_ricker(freq,dt,ampl):
    ts = 2.1/freq  # desired length of wavelet in time is related to peak frequency
    ns = int(ts/dt+0.9999)  # figure out how many time-steps are needed to cover that time
    ts = ns*dt  # and now turn that back into a time that's exactly the required number of steps
    print('Length of wavelet will be %.5f secs (%d steps)' % (ts,ns))
    a2 = (freq*np.pi)**2  # a squared (see equation above)
    t0 = ts/2 - dt/2  # midpoint time of wavelet
    src = np.zeros(ns)
    # create Ricker wavelet (see equation above), offset by time t0 (so midpoint of wavelet is at time t=t0)
    for i in range(ns):
        src[i] = ampl*( (1 - 2*a2*((i*dt-t0)**2)) * np.exp(-a2*((i*dt-t0)**2)) )
    print('Endpoint values are: %.6f %.6f' % (src[0],src[-1]))
    return src,ns

In [ ]:
freq = 8.0  # let's create an 8Hz Ricker source wavelet
src,ns = create_ricker(freq,dt,1.0)

In [ ]:
def plot_source(src):
    plt.figure(figsize=(10,6))
    plt.plot(src) # plot source wavelet
    plt.xlabel('timesteps')
    plt.ylabel('amplitude')
    plt.title('Source Wavelet (Ricker, peak %.1fHz)' % (freq))
    plt.show()

In [ ]:
plot_source(src)

### Check that the maximum frequency in the source wavelet can propagate reliably

For a simple second-order finite-difference, the minimum wavelength of a signal that we can propagate reliably is about 10 cells.

In [ ]:
min_cells_per_wl = 10.0  # minimum of 10 cells per wavelength for reasonably accurate propagation
max_freq = c.min()/(min_cells_per_wl*dx) # calculate the max frequency that can be modelled without numerical dispersion
print('Maximum reliable propagation frequency is about %.1fHz' % max_freq)

#### Want to avoid causing too much dispersion by keeping maximum significant frequency within this limit...

In [ ]:
# plot amplitude spectrum of source wavelet
def plot_ampl_spect(src):
    plt.figure(figsize=(10,6))
    plt.magnitude_spectrum(np.append(src,np.zeros(nt-ns)), Fs=1/dt)  # note padding to nt points
    plt.title('Amplitude Spectrum')
    plt.xlim(0,35)
    plt.show()

In [ ]:
plot_ampl_spect(src)

----
## Create absorbing layers

In [ ]:
# want layer around all edges of the model to be 60 grid-points thick
abswid = 60

Choose one of the following three absorbing models to try (can comment out other two, if desired...)

In [ ]:
# CONSTANT

a = np.zeros((nx,nz))  # initialise array with zeros

a[:abswid,:]  = 1.0  # left layer
a[-abswid:,:] = 1.0  # right layer

a[:,:abswid]  = 1.0  # top layer
a[:,-abswid:] = 1.0  # bottom layer

a[:,-80:]  = 1.0  # uncomment to make bottom layer 80 grid-points instead of 60

absfact = 0.04

In [ ]:
# LINEAR

a = np.zeros((nx,nz))  # initialise array with zeros

# left and right absorbing layers
for i in range(1,abswid+1):
    a[abswid-i,:]   = 1.0*i/abswid  # linear increase towards left boundary
    a[i-abswid-1,:] = 1.0*i/abswid  # linear increase towards right boundary

# top and bottom absorbing layers
for i in range(1,abswid+1):
    for j in range(nx):
        a[j,abswid-i]   = max(a[j,abswid-i],   1.0*i/abswid) # linear increase towards top boundary
        a[j,i-abswid-1] = max(a[j,i-abswid-1], 1.0*i/abswid) # linear increase towards bottom boundary

# uncomment below to make bottom layer 80 grid-points instead of 60...
for i in range(1,81):  # try 80 grid-points in bottom layer
    for j in range(nx):
        a[j,i-81]   = max(a[j,i-81], 1.0*i/80)  # linear increase towards bottom boundary

absfact = 0.08  # without predictive boundary

In [ ]:
# QUADRATIC

a = np.zeros((nx,nz))  # initialise array with zeros

absw2 = abswid*abswid  # useful shorthand

# left and right absorbing layers
for i in range(1,abswid+1):
    a[abswid-i,:]   = 1.0*i*i/absw2  # quadratic increase towards left boundary
    a[i-abswid-1,:] = 1.0*i*i/absw2  # quadratic increase towards right boundary

# top and bottom absorbing layers
for i in range(1,abswid+1):
    for j in range(nx):
        a[j,abswid-i]   = max(a[j,abswid-i],   1.0*i*i/absw2)  # quadratic increase towards top boundary
        a[j,i-abswid-1] = max(a[j,i-abswid-1], 1.0*i*i/absw2)  # quadratic increase towards bottom boundary


# uncomment below to make bottom layer 80 grid-points instead of 60...
for i in range(1,81):  # try 80 grid-points in bottom layer
    for j in range(nx):
        a[j,i-81]   = max(a[j,i-81], 1.0*i*i/(80*80))  # quadratic increase towards bottom boundary

        
absfact = 0.1  # without predictive boundary

#### Show the absorption model

In [ ]:
# various plots so you can check your absorption model...

def plot_absorbing(a):
    plt.figure(figsize=(10,6))
    plt.title('Absorption')

    plt.imshow(a.T) # plotting the absorption as a 2d colour plot (note transposed, to orient plot in way we expect)
    plt.colorbar()
    plt.xlabel('x gridpoints')
    plt.ylabel('z gridpoints')

    plt.show()


def plot_absorb_horiz(a,zgrid):
    plt.figure(figsize=(8,4))
    plt.title('Horizontal x-section at z-gridpoint=%d' % zgrid)

    plt.plot(a[:,zgrid])  # horizontal cross-section through absorption, to show const/linear/quadratic nature
    plt.xlabel('x gridpoints')
    plt.ylabel('absorption coefficient')

    plt.show()


def plot_absorb_vert(a,xgrid):
    plt.figure(figsize=(8,4))
    plt.title('Vertical x-section at x-gridpoint=%d' % xgrid)

    plt.plot(a[xgrid,:])  # vertical cross-section through absorption, to show const/linear/quadratic nature
    plt.xlabel('z gridpoints')
    plt.ylabel('absorption coefficient')

    plt.show()

In [ ]:
plot_absorbing(a)
plot_absorb_horiz(a,200)
plot_absorb_vert(a,200)

Finish by scaling with velocity and other factors, for direct use in simulation later

In [ ]:
absfact = 0.0  # comment this when you switch absorbing layers back on

a[:,:] = a[:,:]*c[:,:]*(dt/dx)*absfact

## Predictive boundary condition

We've already seen this works in 1d by predicting what the value will be at the edge point(s) at the next step, given that we know what the current value is at that point, and at the point just inside, and we know that the wave should be propagating out of the domain at some known speed (since we know the model properties).

By assuming a simple linear connection between the edge point and the point just inside, and then 'shifting' that line across by the crossing factor (i.e. how far the wave should move during the step), we can 'predict' what the new value should be at the edge point.

For the left edge, at $x$=0, this looks like:

$$u(0,t+\delta t)\ \approx\ (1-C_0)\ u(0,t)\ +\ C_0\ u(\delta x,t)\quad
\text{where: }\ C_0 = \frac{c(0)\ \delta t}{\delta x}\ \text{ is 'crossing factor' near $x$=0}$$

And for other edge, at $x$=$L$:

$$u(L,t+\delta t)\ \approx\ (1-C_L)\ u(L,t)\ +\ C_L\ u(L-\delta x,t)\quad
\text{where: }\ C_L = \frac{c(L)\ \delta t}{\delta x} \text{ (i.e. near $x$=$L$)}$$

# However...

**In 2d the above argument isn't quite correct. –Can you see why not...?**

(Answer: the waves may no longer be travelling out of the domain in a direction perpendicular to the boundary.)


### But let's just conveniently ignore that and try applying it anyway... :)

We now have four edges to apply the boundary condition, so...

For the left edge, at $x$=0, this looks like:

$$u(0,z,t+\delta t)\ \approx\ (1-C_{xmin}(z))\ u(0,z,t)\ +\ C_{xmin}(z)\ u(\delta x,z,t)\quad
\text{where: }\ C_{xmin}(z) = \frac{c(0,z)\ \delta t}{\delta x}$$

And for right edge, at $x$=$L$:

$$u(L,z,t+\delta t)\ \approx\ (1-C_{xmax}(z))\ u(L,z,t)\ +\ C_{xmax}(z)\ u(L-\delta x,z,t)\quad
\text{where: }\ C_{xmax}(z) = \frac{c(L,z)\ \delta t}{\delta x}$$

And similar equations for top and bottom.

>**Task 1:**
>
>**Write out the similar equations for top and bottom edges.**
>
>**Use the four expressions for $C_{xmin}$, $C_{xmax}$, $C_{zmin}$, $C_{zmax}$, to give values to four 1d arrays, `C_xmin`, `C_xmax`, `C_zmin`, `C_zmax`, which will be the crossing factors along each of the four edges of the domain.**
>
>**Go to the code cell containing the time-stepping, and fill in code where indicated that uses these four arrays, along with the four approximation equations, to set values for `u_nxt` at the edges of the domain.**  
>
>**When you have it working:**  
>– how do the remaining reflections from the boundaries compare with before having any type of absorbing boundary at all?  
>– how do they compare with the remaining reflections that you saw from the absorbing layer?  
>
>**Switch back on the absorbing layers (comment out the `absfact = 0.0` line in the code cell just above).**  
>– when you run it all again, how does it compare now?
>
>**Now that the predictive boundary is absorbing some of the wavefield at the boundary, we can reduce the strength of the absorbing layers by reducing `absfact`...**  
>– Try halving its value –does that reduce the reflections?  
>– Perhaps try to tweak it even more yourself.  
>– Can you think of more things we could try which might help to 'balance' the two types of absorbing boundaries?

In [ ]:
# FILL IN CODE HERE TO CREATE FOUR 1d ARRAYS C_xmin, C_xmax, C_zmin, C_zmax, and fill them
# with the crossing factors at the four corresponding edges of the domain (task 1)...






# Also added factors below, which allow some tweaking when using predictive along with absorbing layers
# (you can control the amount of predictive at each edge separately by altering the factors below)

# for use without absorbing layers:
predfact_zmin = 1.0
predfact_zmax = 1.0
predfact_xmin = 1.0
predfact_xmax = 1.0

# for use with absorbing layers:
#predfact_zmin = 0.99
#predfact_zmax = 0.98
#predfact_xmin = 0.99
#predfact_xmax = 0.99

# to switch off predictive boundaries:
#predfact_zmin = 0.0
#predfact_zmax = 0.0
#predfact_xmin = 0.0
#predfact_zmax = 0.0

## Line of receivers within domain
–To detect what crosses a particular horizontal line of the domain (i.e. constant depth) over time

In [ ]:
rz = 70  # put receiver line at depth of seventy cells (just above source point)
r = np.zeros((nx,nt))  # an array to store receiver data every step (and to plot later)

----
# Simulation

In [ ]:
# Initialise arrays for wavefields
u = np.zeros((nx,nz)) # current wavefield
u_prv = np.zeros((nx,nz)) # old t-1 wavefield
u_nxt = np.zeros((nx,nz)) # new t+1 wavefield

In [ ]:
# prepare an array to store wavefield snapshots for plotting
snapshot_gap = 10 # set sampling rate used to store wavefield (every 10 timesteps)
wavefield = np.zeros((int(nt/snapshot_gap), nx, nz)) # array to store wavefields every 10 timesteps
print('Storing %d wavefields (every %dth out of %d)' % (wavefield.shape[0],snapshot_gap,nt))

In [ ]:
# a useful variable – shorthand for something that appears regularly in expressions below
dtdx2 = (dt*dt)/(dx*dx)

In [ ]:
def propagate():

    # make sure everything starts off zero
    u[:] = 0.0
    u_prv[:] = 0.0

    # inject first source entry into current wavefield
    u[sx,sz] = src[0]

    # begin time-stepping loop...

    for i in range(nt):
    
        if (i+1)%20==0:  # show progress every 20 steps
            sys.stdout.write('Done step %d (of %d)\r' % (i+1,nt))
    
        # find new wavefield, u_nxt, throughout domain (apart from edges)
        u_nxt[1:-1,1:-1] = ( (2-a[1:-1,1:-1]**2)*u[1:-1,1:-1] - u_prv[1:-1,1:-1]*(1-a[1:-1,1:-1]) \
                           + (c[1:-1,1:-1]**2) * dtdx2 * \
                                (-4*u[1:-1,1:-1]+u[:-2,1:-1]+u[2:,1:-1]+u[1:-1,:-2]+u[1:-1,2:]) ) \
                         / (1+a[1:-1,1:-1])


        # FILL IN CODE HERE FOR PREDICTIVE BOUNDARY CONDITIONS TO SET u_nxt ALONG THE 4 EDGES OF DOMAIN (task 1)...




        # inject source entry for this step at the source point
        if i+1<ns:
            u_nxt[sx,sz] = src[i+1]
    
        # shift wavefields for next time-step
        u_prv[:,:] = u[:,:]
        u[:,:] = u_nxt[:,:]
    
        # fill in values at receivers
        r[:,i] = u[:,rz]
    
        if (i+1)%snapshot_gap == 0: # store the current wavefield u on every tenth step
            wavefield[int((i+1)/snapshot_gap-1)] = u[:,:]

    print('Finished %d time-steps' % (nt))

In [ ]:
propagate()

## Plot wavefield at different times

In [ ]:
def plot_snapshot(plot_time,bounds):
    plt.figure(figsize=(10,7))
    plt.imshow(wavefield[int(plot_time/(dt*snapshot_gap))].T,   # note the wavefield was transposed
               vmin=-bounds, vmax=bounds, cmap='RdBu', interpolation='bilinear')
    plt.title('Wavefield at about %.2fs' % (plot_time))
    plt.colorbar()
    plt.xlabel('x gridpoints')
    plt.ylabel('z gridpoints')
    plt.show()

In [ ]:
plot_snapshot(0.7,0.06)
plot_snapshot(1.3,0.06)

## Plot data at receivers

In [ ]:
def plot_at_receivers(bounds):
    plt.figure(figsize=(10,7))
    plt.imshow(r.T, cmap='RdBu', interpolation='bilinear', aspect='auto', 
               vmin=-bounds, vmax=bounds,   # set bounds for colourmap data
               extent=(0,nx,time,0))  # set bounds for axes
    plt.title('Receiver Data')
    plt.colorbar()
    plt.xlabel('Receiver number')
    plt.ylabel('Time / s')
    plt.show()

In [ ]:
plot_at_receivers(0.06)

## Make a movie! 

In [ ]:
def create_animation(bounds):
    fig = plt.figure(figsize=(10,8))

    plt.title('Wavefield')
    plt.xlabel('x gridpoints')
    plt.ylabel('z gridpoints')

    n = wavefield.shape[0]
    imgs = []
    for i in range(n):
        if i%20==0:  # show progress every 20 frames
            sys.stdout.write('Done %d of %d\r' % (i+1,n))
        img = plt.imshow(wavefield[i].T, vmin=-bounds, vmax=bounds, cmap='RdBu',
                         animated=True, interpolation='bilinear')
        imgs.append([img])

    print('Finished plots for frames, building animation...')

    ani = anim.ArtistAnimation(fig, imgs, interval=50, blit=True)

    plt.close(fig)  # prevent final frame plot from showing up inline below

    return ani

In [ ]:
ani = create_animation(0.06)
print('Preparing HTML (takes a little while...)')
HTML(ani.to_jshtml())